# **LP3_Run1**

In [ ]:
!pip install openai
!pip install python-dotenv
!pip3 install pyomo
!apt install glpk-utils
!pip install glpk

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 52.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 4.3 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
Suggested packages:
  libiodbc2-dev
The following NEW packages will be installed:
  glpk-utils libamd2 libcolamd2 libglpk40 libsuitesparseconfig5
0 upgraded, 5 newly installed, 0 to remove and 45 not upgraded.
Need to get 625 kB of archives.
After this operation, 2,158 kB of additional disk space will be used.
Ge

In [ ]:

import openai
import os
from IPython.display import Markdown
import pyomo


### **Accessing the GPT4 API**

In [ ]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv('api_file.env'))
my_api_key = os.environ['api_key_env']



### **Generate Mathematical Model**

In [ ]:
problem = """The PRODA, S.A. industrial products firm has to face the problem of scheduling
the weekly production of its three products (P1, P2 and P3). These products are
sold to large industrial firms and PRODA, S.A. wishes to supply its products in
quantities that are more profitable for it.

Each product entails three operations contributing to the costs: smelting; mechanisation; assembly and
packaging. The smelting operations for products P1 and P2 could be subcontracted, but the smelting operation for product P3 requires special equipment, thus
preventing the use of subcontracts. PRODA also want to know, how much they should subcontract.

For product P1 the direct unit costs of all possible operations are:
- smelting at PRODA: 0.30$
- subcontracted smelting: 0.50$
- mechanisation: 0.20$
- Assembly and packaging: 0.3$
The unit sales price is 1.50$.

For product P2 the direct unit costs of all possible operations are:
- smelting at PRODA: 0.50$
- subcontracted smelting: 0.60$
- mechanisation: 0.10$
- Assembly and packaging: 0.20$
The unit sales price is 1.80$.

For product P3 the direct unit costs of all possible operations are:
- smelting at PRODA: 0.40$
- mechanisation: 0.27$
- Assembly and packaging: 0.20$
The unit sales price is 1.97$.

Each unit of product P1 requires 6 min of smelting time (if performed at PRODA, S.A.), 6 min of mechanisation time and 3 min of assembly and packaging time, respectively. For product P2, the times are 10, 3 and 2 min, respectively. One unit of product P3 needs 8 min of smelting time, 8 min of mechanisation and 2 min for assembly and packaging. PRODA, S.A. has weekly capacities of 8,000 min of smelting time, 12,000 min of mechanisation time and 10,000 min of assembly and packaging time.
The objective is to maximize weekly profits. """

In [ ]:
client = openai.OpenAI(api_key=os.environ['api_key_env'])

response1 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please formulate a mathematical optimization model for this problem. Include parameters, decision variables, the objective function and the constraints in your answer."},
        {
            "role": "user",
            "content": problem
    }
    ],
    model="gpt-4",
    seed = 1
)


In [ ]:
print(response1.choices[0].message.content)

Parameters:

- Cp1_s: unit cost of smelting for P1 when performed at PRODA
- Cp1_ss: unit cost of subcontracted smelting for P1
- Cp1_m: unit cost of mechanisation for P1 
- Cp1_a: unit cost of assembly and packaging for P1 
- Sp1: unit sales price for P1 

- Cp2_s: unit cost of smelting for P2 when performed at PRODA
- Cp2_ss: unit cost of subcontracted smelting for P2
- Cp2_m: unit cost of mechanisation for P2 
- Cp2_a: unit cost of assembly and packaging for P2 
- Sp2: unit sales price for P2 

- Cp3_s: unit cost of smelting for P3 at PRODA
- Cp3_m: unit cost of mechanisation for P3
- Cp3_a: unit cost of assembly and packaging for P3
- Sp3: unit sales price for P3

- Time_s: Total smelting time available
- Time_m: Total mechanisation time available
- Time_a: Total assembly and packaging time available

- TimeP1_s, TimeP1_m, TimeP1_a: required time by P1 to perform smelting, mechanisation and assembly
- TimeP2_s, TimeP2_m, TimeP2_a: required time by P2 to perform smelting, mechanisat

### **Generate Pyomo Code**

In [ ]:

response2 = client.chat.completions.create(
    messages=[
        {"role": "system", "content": "Please write a python pyomo code for this optimization problem. Use sample data where needed. Indicate where you use sample data."},
        {
            "role": "user",
            "content": response1.choices[0].message.content
        }
    ],
    model="gpt-4",
    seed = 1
)

In [ ]:
Markdown(response2.choices[0].message.content)

Here's the Pyomo code for the above optimization problem:

```python
from pyomo.environ import *

# create a model
model = ConcreteModel()

# These numbers are made for demo purposes. Change these numbers accordingly.
Cp1_s, Cp1_ss, Cp1_m, Cp1_a, Sp1 = 4, 5, 6, 7, 8
Cp2_s, Cp2_ss, Cp2_m, Cp2_a, Sp2 = 9, 10, 11, 12, 13
Cp3_s, Cp3_m, Cp3_a, Sp3 = 14, 15, 16, 17

Time_s, Time_m, Time_a = 20, 30, 40

TimeP1_s, TimeP1_m, TimeP1_a = 3, 4, 5
TimeP2_s, TimeP2_m, TimeP2_a = 6, 7, 8
TimeP3_s, TimeP3_m, TimeP3_a = 9, 10, 11

# declare decision variables
model.P1 = Var(domain=NonNegativeReals)
model.P1s = Var(domain=NonNegativeReals)
model.P2 = Var(domain=NonNegativeReals)
model.P2s = Var(domain=NonNegativeReals)
model.P3 = Var(domain=NonNegativeReals)

# declare objective
model.profit = Objective(
    expr = Sp1*(model.P1 + model.P1s) - (Cp1_s*model.P1 + Cp1_ss*model.P1s + Cp1_m*model.P1 + Cp1_a*model.P1)+ 
    Sp2*(model.P2 + model.P2s) - (Cp2_s*model.P2 + Cp2_ss*model.P2s + Cp2_m*model.P2 + Cp2_a*model.P2) +
    Sp3*model.P3 - (Cp3_s*model.P3 + Cp3_m*model.P3 + Cp3_a*model.P3),
    sense=maximize)

# declare constraints
model.time_s = Constraint(expr = model.P1*TimeP1_s + model.P2*TimeP2_s + model.P3*TimeP3_s <= Time_s)
model.time_m = Constraint(expr = model.P1*TimeP1_m + model.P2*TimeP2_m + model.P3*TimeP3_m <= Time_m)
model.time_a = Constraint(expr = model.P1*TimeP1_a + model.P2*TimeP2_a + model.P3*TimeP3_a <= Time_a)

# choose the solver
solver = SolverFactory('glpk')

# solve the model
solver.solve(model)

# print the results
print("P1 = ", model.P1())
print("P1s = ", model.P1s())
print("P2 = ", model.P2())
print("P2s = ", model.P2s())
print("P3 = ", model.P3())
```

In this script, the `ConcreteModel` class is used to create a mathematical model that includes variables, objectives, and constraints. The model, objective and constraints are specified in a declarative manner, which allows the solver to manipulate the expressions directly. The `SolverFactory` class is used to specify the solver that will be used to solve the model.

The important thing to note here is the made up numbers I used for demo purpose. All of these data should be replaced by real data in an actual scenario.

This script assumes that GLPK is installed in the system which may not be the case for all users. Depending upon the installed solvers, suitable solver can be used.

### **Run the code Generated by GPT4**

In [ ]:
from pyomo.environ import *

# create a model
model = ConcreteModel()

# START - correct input data added bu human
Cp1_s = 0.30  # unit cost of smelting for P1 at PRODA
Cp1_ss = 0.50  # unit cost of subcontracted smelting for P1
Cp1_m = 0.20  # unit cost of mechanisation for P1
Cp1_a = 0.30  # unit cost of assembly and packaging for P1
Sp1 = 1.50  # unit sales price for P1

Cp2_s = 0.50  # unit cost of smelting for P2 at PRODA
Cp2_ss = 0.60  # unit cost of subcontracted smelting for P2
Cp2_m = 0.10  # unit cost of mechanisation for P2
Cp2_a = 0.20  # unit cost of assembly and packaging for P2
Sp2 = 1.80  # unit sales price for P2

Cp3_s = 0.40  # unit cost of smelting for P3 at PRODA
Cp3_m = 0.27  # unit cost of mechanisation for P3
Cp3_a = 0.20  # unit cost of assembly and packaging for P3
Sp3 = 1.97  # unit sales price for P3

Time_s = 8000  # Total smelting time available in minutes
Time_m = 12000  # Total mechanisation time available in minutes
Time_a = 10000  # Total assembly and packaging time available in minutes

TimeP1_s = 6  # required time by P1 to perform smelting in minutes
TimeP1_m = 6  # required time by P1 to perform mechanisation in minutes
TimeP1_a = 3  # required time by P1 to perform assembly in minutes

TimeP2_s = 10  # required time by P2 to perform smelting in minutes
TimeP2_m = 3  # required time by P2 to perform mechanisation in minutes
TimeP2_a = 2  # required time by P2 to perform assembly in minutes

TimeP3_s = 8  # required time by P3 to perform smelting in minutes
TimeP3_m = 8  # required time by P3 to perform mechanisation in minutes
TimeP3_a = 2  # required time by P3 to perform assembly in minutes

#END
# declare decision variables
model.P1 = Var(domain=NonNegativeReals)
model.P1s = Var(domain=NonNegativeReals)
model.P2 = Var(domain=NonNegativeReals)
model.P2s = Var(domain=NonNegativeReals)
model.P3 = Var(domain=NonNegativeReals)

# declare objective
model.profit = Objective(
    expr = Sp1*(model.P1 + model.P1s) - (Cp1_s*model.P1 + Cp1_ss*model.P1s + Cp1_m*model.P1 + Cp1_a*model.P1)+
    Sp2*(model.P2 + model.P2s) - (Cp2_s*model.P2 + Cp2_ss*model.P2s + Cp2_m*model.P2 + Cp2_a*model.P2) +
    Sp3*model.P3 - (Cp3_s*model.P3 + Cp3_m*model.P3 + Cp3_a*model.P3),
    sense=maximize)

# declare constraints
model.time_s = Constraint(expr = model.P1*TimeP1_s + model.P2*TimeP2_s + model.P3*TimeP3_s <= Time_s)
model.time_m = Constraint(expr = model.P1*TimeP1_m + model.P2*TimeP2_m + model.P3*TimeP3_m <= Time_m)
model.time_a = Constraint(expr = model.P1*TimeP1_a + model.P2*TimeP2_a + model.P3*TimeP3_a <= Time_a)

# choose the solver
solver = SolverFactory('glpk')

# solve the model
solver.solve(model)

# print the results
print("P1 = ", model.P1())
print("P1s = ", model.P1s())
print("P2 = ", model.P2())
print("P2s = ", model.P2s())
print("P3 = ", model.P3())

P1 =  None
P1s =  None
P2 =  None
P2s =  None
P3 =  None


### **Edit and Run the code for the mathematical model produced by GPT4 (Circumstantial)**

In [ ]:
results = solver.solve(model)
results

{'Problem': [{'Name': 'unknown', 'Lower bound': -inf, 'Upper bound': inf, 'Number of objectives': 1, 'Number of constraints': 3, 'Number of variables': 5, 'Number of nonzeros': 9, 'Sense': 'maximize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'unbounded', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.02271103858947754}]}